In [ ]:
from datasets import load_dataset

In [9]:
# fatto ciò provare a usare i dataset delle batterie

!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz -P ./dataset/
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz -P ./dataset/


# carico il dataset in json unendo training e validation
# la funzione load automaticamente decomprime da .gz
data_files = {"train": "dataset/SQuAD_it-train.json.gz", "test": "dataset/SQuAD_it-test.json.gz"} 
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

--2025-03-11 12:12:28--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2025-03-11 12:12:28--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: ‘./dataset/SQuAD_it-train.json.gz.1’

SQuAD_it-train.json 100%[===================>]   7.37M  10.9MB/s    in 0.7s    

2025-03-11 12:12:29 (10.9 MB/s) - ‘./dataset/SQuAD_it-train.j

Generating train split: 442 examples [00:00, 454.68 examples/s]
Generating test split: 48 examples [00:00, 611.74 examples/s]


DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [11]:
# direttamente da cartella remota

url = "https://github.com/crux82/squad-it/raw/master/" 
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

SyntaxError: invalid syntax (2726083861.py, line 3)

## SLICE and DICE

In [1]:
from datasets import load_dataset

!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip" -P ./dataset/
!unzip "dataset/drugsCom_raw.zip" -P ./dataset/
data_files = {"train": "dataset/drugsComTrain_raw.tsv", "test": "dataset/drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--2025-03-12 09:52:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘./dataset/drugsCom_raw.zip.3’

drugsCom_raw.zip.3      [        <=>         ]  41.00M  8.97MB/s    in 5.5s    

2025-03-12 09:53:02 (7.44 MB/s) - ‘./dataset/drugsCom_raw.zip.3’ saved [42989872]

Archive:  dataset/drugsCom_raw.zip
caution: filename not matched:  -P
caution: filename not matched:  ./dataset/


In [33]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Peek at the first few examples
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

## 3 problemi:
 1. unnamed 0 cos'è?
 2. condition labels mix tra uppercase e lowercase
 3. review pieno di refusi tra caratteri pyhton 3 html

######   SOluzioni:

In [ ]:
#1.
# questo unnamed 0 cosa vuole dire?, controlliamo che sia un id unico per paziente



for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

# hanno la stessa lunghezza, questo significa che è un campo univoco, diciamo che è l'id
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [35]:
# 2.
#normalizziamo le condition labels, tutte in minuscolo, utilizzando .map

#def lowercase_condition(example):
#    return {"condition": example["condition"].lower()}
#  drug_dataset.map(lowercase_condition)

# ci sono dei noneType, quindi dobbiamo filtrarli

# def filter_nones(x):
#     return x["condition"] is not None   # ritorna true (!= da None) o false  

# drug_dataset.filter(filter_nones)   # filter è molto simile a .map (true lo tiene, false lo scarta)

# possiamo farlo utillizando una lambda function (lambda <arguments> : <expression>)
# es (lambda x: x * x)(3) = 9
# es (lambda base, height: 0.5 * base * height)(4, 8) = 16

drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)     # uguale a filter_nones!!!!!
# tolte i None, normalizziamo a lowercase
drug_dataset = drug_dataset.map(lambda x : {"condition": x["condition"].lower()} ) # uguale a lowercase_condition
# Check that lowercasing worked
drug_dataset["train"]["condition"][:3]


Map: 100%|██████████| 53471/53471 [00:02<00:00, 26455.83 examples/s]


['left ventricular dysfunction', 'adhd', 'birth control']

In [36]:
#2.5

# creiamo una colonna con la lunghezza delle review (in numero di parole)

def compute_review_length(example):
    return {"review_length": len(example["review"].split())}
drug_dataset = drug_dataset.map(compute_review_length)        #.map chiama la funzione su ogni elemento e aggiunge l'elemento(perchè non lo trova, altrimenti lo modificherebbe)

print(drug_dataset.num_rows)
# eliminiamo i campi con le review < 30
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

Map: 100%|██████████| 53471/53471 [00:01<00:00, 30459.80 examples/s]


{'train': 160398, 'test': 53471}


Filter: 100%|██████████| 53471/53471 [00:00<00:00, 343002.61 examples/s]

{'train': 138514, 'test': 46108}


In [31]:
# Use the Dataset.sort() function to inspect the reviews with the largest numbers of words. See the documentation to see which argument you need to use sort the reviews by length in descending order.
# print(drug_dataset.sort("review", reverse=True)[0])

KeyboardInterrupt: 

In [ ]:
# 3. 
# sistemare caratteri html

import html
%time drug_dataset = drug_dataset.map(       lambda x: {"review": html.unescape(x["review"])},  batched=True, num_proc = 2   )    # per ogni elemento, lo aggiorna con la nuova review non-html

# batched=True  serve a mandare un batch alla volta alla funzione, e non un elemento alla volta, per velocizzare!
# num_proc= 8   serve a parellelizzare 


# quindi al posto di loopare su ogni elemento, utilizza la list comprehension(?) su ogni batch
# NB questa cosa viene utilizzata anche per il TOKENIZATION




Map (num_proc=2): 100%|██████████| 53766/53766 [00:00<00:00, 471414.34 examples/s]

CPU times: user 28.1 ms, sys: 35.2 ms, total: 63.3 ms
Wall time: 346 ms


In [ ]:
### return_overflowing_tokens=True
# serve a troncare a tot, ma tenere anche il resto dell'input(troncandolo eventualmente)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=9,
        return_overflowing_tokens=True,
    )

# result = tokenize_and_split(drug_dataset["train"][0])
# [len(inp) for inp in result["input_ids"]]

# per utilizzare la funzione dobbiamo rimuovere le colonne derivate dal vecchio dataset = drug_dataset["train"].column_names.
# questo perchè altrimenti il dataset di partenza e finale hanno lunghezze diverse e a .map non piace
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names)
len(tokenized_dataset["train"]), len(drug_dataset["train"])

tokenized_dataset.column_names


Map: 100%|██████████| 53766/53766 [00:14<00:00, 3635.69 examples/s]


{'train': ['input_ids',
  'token_type_ids',
  'attention_mask',
  'overflow_to_sample_mapping'],
 'test': ['input_ids',
  'token_type_ids',
  'attention_mask',
  'overflow_to_sample_mapping']}

In [4]:
# oppure possiamo rendere più lungo il dataset di partenza.

def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

#usando questo mapping manteniamo le colonne.
# se queste colonne ci servono per un eventuale post-processing, dobbiamo usare questo approccio.

tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset.column_names

NameError: name 'drug_dataset' is not defined

### From Dataset s to DataFrame s and back

In [55]:
drug_dataset.reset_format()         # resetto il formato

print("prime 3 righe del dataset")
print(drug_dataset["train"][:3])    # print as a dict

print("\n prime 3 righe del dataset in formato pandas")
drug_dataset.set_format("pandas")   # cambio in pandas  (è solo il formato riotrnato di quando viene chiamato, resta un dataset!!!)
print(drug_dataset["train"][:3])    # print in pandas version

train_df = drug_dataset["train"][:]  # creo un pandas.Dataframe dall'intero train_set (prima era solo visto come un pandas, ora train_df è un pandas)

# ora posso fare tutte le operazioni possibili in un pandas
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "condition": "frequency"})
)

print("\n head del pandas frequencies")
print(frequencies.head())

from datasets import Dataset
freq_dataset = Dataset.from_pandas(frequencies)   # creo un dataset da un pandas
print("\n stampa del dataframe derivato dal pandas frequencies")
print(freq_dataset)         




prime 3 righe del dataset
{'patient_id': [206461, 95260, 92703], 'drugName': ['Valsartan', 'Guanfacine', 'Lybrel'], 'condition': ['Left Ventricular Dysfunction', 'ADHD', 'Birth Control'], 'review': ['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"', '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far th

### create a validation set

In [3]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

NameError: name 'drug_dataset' is not defined

## Saving a dataset

### 3 Methods
Data format     Function
Arrow           Dataset.save_to_disk()
CSV	            Dataset.to_csv()
JSON	        Dataset.to_json()


In [2]:
from datasets import load_from_disk

drug_dataset_clean.save_to_disk("dataset/drug-reviews")   # save evrey split in a different folder


drug_dataset_reloaded = load_from_disk("dataset/drug-reviews")  # load from the main folder
drug_dataset_reloaded



# For the CSV and JSON formats, we have to store each split as a separate file. 
# One way to do this is by iterating over the keys and values in the DatasetDict object:
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"dataset/drug-reviews-{split}.jsonl")    #save json


data_files = {
    "train": "drug-reviews-train.jsonl",
    "validation": "drug-reviews-validation.jsonl",
    "test": "drug-reviews-test.jsonl",
}
drug_dataset_reloaded = load_dataset("json", data_files=data_files)   #load json


/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'drug_dataset_clean' is not defined

In [53]:
from datasets import load_dataset
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader
import torch
from tqdm.auto import tqdm

#!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip" -P ./dataset/
#!unzip "dataset/drugsCom_raw.zip" -P ./dataset/
data_files = {"train": "dataset/drugsComTrain_raw.tsv", "test": "dataset/drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")


#rinomino Unnamed con id hanno la stessa lunghezza, questo significa che è un campo univoco, diciamo che è l'id
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)

#### condition tutto minuscolo ####
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)     # uguale a filter_nones!!!!!
# tolte i None, normalizziamo a lowercase
drug_dataset = drug_dataset.map(lambda x : {"condition": x["condition"].lower()} ) # uguale a lowercase_condition
# Check that lowercasing worked
drug_dataset["train"]["condition"][:10]

### sistem html in drug review
import html
drug_dataset = drug_dataset.map( lambda x: {"review": html.unescape(x["review"])},  batched=True, num_proc = 2   )    # per ogni elemento, lo aggiorna con la nuova review non-html
drug_dataset = drug_dataset.map( lambda x: {"condition": html.unescape(x["condition"])},  batched=True, num_proc = 2   )
### creo validation

drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset = drug_dataset_clean

print(drug_dataset)

## tokenizzo rimuovendo tutte le colonne in più

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result
    

tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

number_of_labels = len(set(tokenized_dataset["train"]["condition"]))
number_of_labels

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ELIMIARE LE COLONNE (e altro )NEL DATASET CHE NON CI INTERESSANO (La classe trainer lo fa in automatico)

tokenized_datasets = tokenized_dataset.remove_columns(["patient_id", "drugName", "review", "rating", "date", "usefulCount"])
tokenized_datasets = tokenized_datasets.rename_column("condition", "labels")
tokenized_datasets.set_format("torch")
print("colonne rimaste nel dataset:")
print(tokenized_datasets["train"].column_names)

# creo una mappatura labels id e poi lo applico. il modello vuole solo numeri 
unique_conditions = list(set(tokenized_datasets["train"]["labels"]))
condition_to_id = {cond: idx for idx, cond in enumerate(unique_conditions)}

# per fare veloce, rimuovo le condiizoni non presenti nel train test
filtered_test_set = tokenized_datasets["test"].filter(lambda x: x["labels"] in condition_to_id)
tokenized_datasets["test"] = filtered_test_set

# per fare veloce, rimuovo le condiizoni non presenti nel train e validation test
filtered_test_set = tokenized_datasets["validation"].filter(lambda x: x["labels"] in condition_to_id)
tokenized_datasets["validation"] = filtered_test_set

# # Troviamo tutte le condizioni nel test set
# test_conditions = set(tokenized_datasets["validation"]["labels"])

# # Troviamo le condizioni che NON sono nel training set
# unknown_conditions = test_conditions - set(condition_to_id.keys())

# # Stampiamo eventuali condizioni sconosciute
# if unknown_conditions:
#     print("⚠️ Attenzione! Le seguenti condizioni sono presenti nel test set ma non nel training set:")
#     print(len(unknown_conditions))
# else:
#     print("✅ Tutte le condizioni del test set esistono nel training set.")

def encode_labels(example):
    return {"labels": condition_to_id[example["labels"]]}

tokenized_datasets = tokenized_datasets.map(encode_labels)

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 128318
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 32080
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53471
    })
})
colonne rimaste nel dataset:
['labels', 'input_ids', 'token_type_ids', 'attention_mask']
✅ Tutte le condizioni del test set esistono nel training set.


Map: 100%|██████████| 78232/78232 [00:09<00:00, 7850.19 examples/s]


In [60]:
# DEFINIAMO IL DATALOADER

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8,collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8,collate_fn=data_collator
)
for batch in train_dataloader: # CHECK CORRECTENESS OF DATA
    break
{k: v.shape for k, v in batch.items()}


# INSTANZIO IL MODELLO
checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=number_of_labels)
outputs = model(**batch)   # CONTROLLLO CHE TUTTO È OK passando il batch (batch da 8, 2 logits)
print(outputs.loss, outputs.logits.shape)


            #### CREIAMO IL TRAINING ####

# OTTIMIZZATORE E LR SCHEDULER, COME QUELLI DEL TRAINING USATO SOPRA
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

# SCELGO LA gpu COME DEVICE
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

# THE LOOP TRAINING
progress_bar = tqdm(range(num_training_steps))

model.train()   # SETTA IL MODELLO PER ESSERE TRAINATO
for epoch in range(num_epochs):   # TUTTO IL DATALOADER PER TOT EPOCHE
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


tensor(6.8221, grad_fn=<NllLossBackward0>) torch.Size([8, 838])
70515
cuda


 37%|███▋      | 26117/70515 [14:13<24:07, 30.68it/s]

In [ ]:
# metriche?

In [ ]:
# Summarization pipeline
summ = pipeline("summarization")
for review in tokenized_datasets["train"]["labels"]:
    print(summ( review ))